<a href="https://colab.research.google.com/github/robertmaxwilliams/talking-statues/blob/master/colab/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## IMPORTANT
Before you run this notebook you need to copy the static files over!
foo.html
index.css
corpus.txt

## Imports

In [1]:
# !pip install gpt_2_simple bottle markovify flask-ngrok flask==0.12.2

In [2]:
import gpt_2_simple as gpt2
import os
import requests
from random import randint as dice
import time
import markovify

class bcolors:
    HEADER = "\033[95m"
    OKBLUE = "\033[94m"
    OKGREEN = "\033[92m"
    WARNING = "\033[93m"
    FAIL = "\033[91m"
    ENDC = "\033[0m"
    BOLD = "\033[1m"
    UNDERLINE = "\033[4m"

sess = None

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## Downloading model, starting session, and defining generation function

In [3]:
model_name = "124M"
if not os.path.isdir(os.path.join("models", model_name)):
    print(f"Downloading {model_name} model...")
    gpt2.download_gpt2(model_name=model_name)

In [4]:
!mkdir checkpoint && cp -r models/124M checkpoint/run1

mkdir: cannot create directory ‘checkpoint’: File exists


In [5]:
# Restart session if running this cell again
if (sess != None):
    gpt2.reset_session(sess)

sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, multi_gpu=False)
# gpt2.load_gpt2(sess, multi_gpu=True)

def generate_text(prefix, length, num_samples):
    prefix = prefix[:length]
    output = gpt2.generate(
        sess,
        prefix=prefix,
#         include_prefix=False,
        include_prefix=True,
        return_as_list=True,
        length=length,
        nsamples=num_samples,
#         batch_size=num_samples,
    )
    output = [x[len(prefix):] for x in output]
    return output


Loading checkpoint checkpoint/run1/model.ckpt
INFO:tensorflow:Restoring parameters from checkpoint/run1/model.ckpt


## Creating and Starting server

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, send_from_directory, request
import time

app = Flask(__name__)
# run_with_ngrok(app) 

# build text model
with open("../static/corpus.txt", encoding="utf8") as f:
    corpus = f.read()

text_model = markovify.Text(corpus)


def generate_from_text_model(text, length=100, num_samples=1):
    gems = generate_text(text, length, num_samples)
    ret = ""
    try:
        for i in range(num_samples):
            ret += f"<div class='predictionBox'> <p><pre>{gems[i]}</pre></p> </div>"
    except Exception as e:
        print(f"FEEE\n{e}\nEEEEF")
        return "sorry, model failure: " + str(e)
    return ret


def random_color():
    return "{:06x}".format(dice(0, 0xFFFFFF))


def colorize(text):
    words = text_model.word_split(text)
    ret = ""
    for w in words:
        ret += f"<span style='background-color:#{random_color()}'>{w}</span>"
    return ret


@app.route("/static/<filename>")
def server_static(filename):
    return send_from_directory("../static/", filename)


@app.route("/")
def index():
    return send_from_directory("../static/", "foo.html")


@app.route("/generate", methods=["POST"])
def generate():
    generate_start= time.time()
    
    text = request.form["text"]
    print(f"Generate: prefix = {text}")
    stories = generate_from_text_model(text, 20, num_samples=1)
    
    generate_end = time.time()
    print(f"Time to respond: {generate_end - generate_start}")
    
    return stories


@app.route("/highlight", methods=["POST"])
def highlight():
    time.sleep(0.5)
    text = request.form["text"]
    print(f"Highlight: text = {text}")
    return "<p>" + colorize(text) + "</p>"

def print_red(text): 
    print("\033[91m {}\033[00m".format(text))

@app.route("/webhook", methods=["POST"])
def webhook():
    print("WEbhhook called!")
#     body = request.body.getvalue().decode('utf-8')
#     print_red(body)
#     body = json.loads(body)
#     print(f'request:\n {request.data}')
#     print(f'json:\n {request.data.json}')
    body = request.get_json()
    
    query_text = body["queryResult"]["queryText"] 
    print(f'Query_text:\t{query_text}')
    if query_text == "GOOGLE_ASSISTANT_WELCOME":
        print("Welcome page, returning empty json object")
        return '{}'
    
    parameters = body['queryResult']['parameters']
    print(f'Params:\t{parameters}')
          
    story = 'Let me tell you a story. '
    if parameters['subject_thing'] != '':
        story += f"There was a {parameters['subject_thing']}. "
    if parameters['subject_place'] != '':
        story += f"This took place at {parameters['subject_place']}. "
    if parameters['subject_event'] != '':
        story += f"It was a great day, the day of a {parameters['subject_event']}. "
    story += 'THE PLOT!'
    print(f'Story:\t{story}')
    json_text = f'{{"fulfillmentText": "{story}"}}'
    return json_text

app.run(host="0.0.0.0", port=5000, ssl_context='adhoc')


 * Running on https://0.0.0.0:5000/ (Press CTRL+C to quit)
35.236.232.76 - - [30/Mar/2020 22:55:08] "POST /webhook HTTP/1.1" 200 -


WEbhhook called!
Query_text:	a duck
Params:	{'subject_thing': 'duck', 'subject_event': '', 'subject_place': ''}
Story:	Let me tell you a story. There was a duck. THE PLOT!
